In [1]:
pip install requests transformers beautifulsoup4


In [2]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
import requests

from bs4 import BeautifulSoup

# Function to retrieve news sentiment from the API
def get_news_sentiment(ticker, api_key, limit=50, sort='LATEST'):
    # Base URL for the API
    url = 'https://www.alphavantage.co/query'

    # Parameters for the API request
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': ticker,
        'limit': limit,
        'sort': sort,
        'apikey': api_key
    }

    # Send the GET request
    response = requests.get(url, params=params)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()

        # Return the data if successful
        return data
    else:
        # If the API call was not successful, return None or error message
        print(f"Error fetching data: {response.status_code}")
        return None

# Function to extract and clean text content from a URL
def get_article_text(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find relevant HTML tags (headlines and paragraphs)
    results = soup.find_all(['h1', 'p'])
    text = [result.get_text() for result in results]

    # Combine the text into a single article
    ARTICLE = " ".join(text)
    return ARTICLE

# Function to prepare text for summarization and summarize it
def summarize_text(text, summarizer):
    # Split text into sentences and prepare chunks for summarization
    ARTICLE = text.replace('.', '.<eos>').replace('!', '!<eos>').replace('?', '?<eos>')
    sentences = ARTICLE.split('<eos>')

    # Chunk the article text if it's too large for the summarizer
    max_chunk = 500
    chunks = []
    current_chunk = 0
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    # Combine the chunks into strings
    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    # Run the summarizer on the chunks
    summarized = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

    # Combine the summaries into one text
    summary_text = ' '.join([summ['summary_text'] for summ in summarized])

    return summary_text

# Function to display and summarize the news articles
def display_and_summarize_news(news_data, summarizer):
    if 'feed' in news_data:
        articles = news_data['feed']
        if articles:
            for i, article in enumerate(articles[:3]):  # Limit to first 3 articles for demo
                #print(f"\nArticle {i+1}: {article['title']}")
                #print(f"Source: {article['source']}")
                #print(f"Published: {article['time_published']}")


                # Fetch the full article text from the URL
                article_url = article['url']
                full_text = get_article_text(article_url)

                # Summarize the article text
                summary = summarize_text(full_text, summarizer)

                # Display the summary
                #print(f"Summary: {summary}\n")
                string = f"\nArticle {i+1}: {article['title']}\nSource: {article['source']}\nPublished: {article['time_published']}\nSummary: {summary}\n"
                print(string)
        else:
            print("No news articles found for this ticker.")
    else:
        print("Invalid response structure.")

# Main function
if __name__ == "__main__":
    # Input from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL for Apple): ")
    api_key = '-------'  # Replace with your actual API key

    # Initialize the summarizer from Huggingface's pipeline
    #summarizer = pipeline("summarization")

    # Fetch the news sentiment data
    news_data = get_news_sentiment(ticker, api_key)

    # If the news data is retrieved, summarize and display it
    if news_data:
        display_and_summarize_news(news_data, summarizer)
    else:
        print("Failed to retrieve news data.")


Enter the stock ticker symbol (e.g., AAPL for Apple): aapl

Article 1: Archegos Capital's Bill Hwang Receives 18 Year Sentence For Fraud, Market Manipulation - UBS Gr  ( NYSE:UBS ) 
Source: Benzinga
Published: 20241120T220835
Summary:  Archegos Capital Management founder Bill Hwang was sentenced on Wednesday to 18 years in prison for fraud and market manipulation tied to the fund's dramatic collapse in 2021 . Hwang's lawyers had asked that he be given no prison time at all and Hwang asked the judge to impose a sentence that would allow him to continue to serve society . The collapse contributed to the demise of Credit Suisse Group AG and rippled through Wall Street causing significant losses .



Your max_length is set to 120, but your input_length is only 53. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)



Article 2: Qualcomm's Shift From Handsets To AI And Automotive Wins Analysts' Confidence - Qualcomm  ( NASDAQ:QCOM ) 
Source: Benzinga
Published: 20241120T200730
Summary:  Qualcomm plans to shift revenue reliance from handsets to higher-growth segments like Automotive, IoT, PCs, XR/AR, and Industrial applications . Several Wall Street analysts reiterated Qualcomm Inc QCOM after the company’s 2024 investor day .  Qualcomm presented a highly attended investor day focused on its diversification strategy first outlined in 2021, emphasizing accelerated adoption of its platforms driven by AI . Management expressed confidence in its ability to sustain growth despite these challenges . The anticipated impact of Apple’s internal modem transition poses a significant headwind .  Join Benzinga Edge and unlock all the major upgrades, downgrades and changes to the market's most accurate analysts . Trade confidently with insights and alerts from analyst ratings, free reports and breaking news that a

Your max_length is set to 120, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)



Article 3: Earnings Data Deluge
Source: Zacks Commentary
Published: 20241120T154800
Summary:  CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery . Please submit your best shots of our featured destinations for next week . Visit CNN iReport.com/Travel next Wednesday for a new gallery of snapshots .

